In [1]:
%%capture
!pip install checklist

In [2]:
%%capture
!pip install transformers

In [3]:
import torch
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [4]:
import checklist
from checklist.editor import Editor
from checklist.test_types import MFT, INV, DIR
from checklist.pred_wrapper import PredictorWrapper
from checklist.test_suite import TestSuite
from checklist.expect import Expect
from checklist.perturb import Perturb
import warnings
warnings.filterwarnings('ignore')

In [5]:
editor = Editor()

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

config = AutoConfig.from_pretrained("IsaacRodgz/Tamil-Hate-Speech")
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base", use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained("IsaacRodgz/Tamil-Hate-Speech", config=config)
model = model.to(device)

Downloading:   0%|          | 0.00/922 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

In [7]:
def predict(text):
    # tokenize the input data and move to GPU if available
    model_inputs = tokenizer(
        text,
        truncation=True,
        max_length=256,
        return_tensors='pt'
    ).to(device)
    
    # Model returns the raw logtits
    logits = model(input_ids=model_inputs['input_ids'], attention_mask=model_inputs['attention_mask'])['logits']
    
    # Transform logits into actual probabilities
    probs = torch.nn.functional.softmax(logits, dim=1)
    pred_prob = probs.cpu().detach().numpy().tolist()[0]
    # Get index with highest probability, move to cpu and transform from pytorch tensor to numpy array and finally from numpy.int64 to int
    #pred_index = int(probs.argmax(dim=1).cpu().detach().numpy()[0])
    
    # Get label associated to index
    #pred_label = model.config.id2label[pred_index]
        
    return pred_prob

In [142]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('Behavioral Testing')

In [143]:
data = worksheet.worksheets()[3].get_all_values()[2:]
data_df = pd.DataFrame.from_records(data)
data_df.columns = ["Tamil", "Index", "Label"]
df_groups = data_df.groupby('Index')

In [144]:
data_df.head(5)

,Tamil,Index,Label
0,விவசாயத்தை மதிக்கும் உங்கள் குடும்பத்திற்கு மு...,0,Non-Hate-Speech
1,விவசாயத்தை மதிக்கும் உங்கள் குடும்பத்திற்கு மு...,0,Non-Hate-Speech
2,"பெத்த புள்ளையே தாய்,தகப்பன் தேவையில்லாத தொல்லை...",0,Non-Hate-Speech
3,"பெத்த புள்ளையே தாய்,தகப்பன் தேவையில்லாததொ ல்லை...",0,Non-Hate-Speech
4,இந்த குடும்பத்தை அழித்தொழிக்கபடவேன்டும்,1,Hate-Speech


In [145]:
import numpy as np
def predict_tamilHS(input):
  p1 = np.array([predict(x)[1] for x in input]).reshape(-1, 1)
  p0 = 1 - p1
  return np.hstack((p0,p1))

In [146]:
from checklist.pred_wrapper import PredictorWrapper
wrapped_pp = PredictorWrapper.wrap_softmax(predict_tamilHS)

INV for typos

In [147]:
test = MFT(
    data_df['Tamil'].tolist(), labels=[int(i) for i in data_df['Index']], name="Robustness to typos", capability="Typos"
)

MFT - Adversarial

In [148]:
data1 = worksheet.worksheets()[2].get_all_values()[2:]
data_df1 = pd.DataFrame.from_records(data1)
data_df1.columns = ["Tamil", "English","Index"]


In [149]:
test1 = MFT(
    data_df1['Tamil'].tolist(), labels=[int(i) for i in data_df1['Index']], name="Adversarial input", capability="Adversarial"
)

For MFT - Neutral data

In [150]:
data2 = worksheet.worksheets()[1].get_all_values()[2:]
data_df2 = pd.DataFrame.from_records(data2)
data_df2.columns = ["Tamil", "English","Index"]


In [151]:
test2 = MFT(
    data_df2['Tamil'].tolist(), labels=[int(i) for i in data_df2['Index']], name="Neutral input", capability="neutral"
)

In [152]:
data3 = worksheet.worksheets()[0].get_all_values()[2:]
data_df3 = pd.DataFrame.from_records(data3)
data_df3.columns = ["Tamil", "Tamil latin","Tanglish","English","Lables","Index"]

In [153]:
tamilTest = MFT(
    data_df3['Tamil'].tolist(), labels=[int(i) for i in data_df3['Index']], name="Tamil input", capability="language input"
)
tamilLatinTest = MFT(
    data_df3['Tamil latin'].tolist(), labels=[int(i) for i in data_df3['Index']], name="Tamil latin input", capability="language input"
)
tanglishTest = MFT(
    data_df3['Tanglish'].tolist(), labels=[int(i) for i in data_df3['Index']], name="Tanglish input", capability="language input"
)

Creating test suite

In [154]:
suite = TestSuite()
suite.add(test,capability = "Typos--")
suite.add(test1,capability = "Adversarial--")
suite.add(test2,capability = "Neutral--")
suite.add(tamilTest,capability = "MFT-tamil")
suite.add(tamilLatinTest,capability = "MFT-tamillatin")
suite.add(tanglishTest,capability = "MFT-tanglish")
suite.run(wrapped_pp,overwrite=True)

Running Robustness to typos
Predicting 11 examples
Running Adversarial input
Predicting 16 examples
Running Neutral input
Predicting 19 examples
Running Tamil input
Predicting 12 examples
Running Tamil latin input
Predicting 12 examples
Running Tanglish input
Predicting 12 examples


In [155]:
%%capture cap
type(suite.summary())

In [156]:
f = open("suiteOutput.txt", "w") 
print(cap, file=f)
f.close()